<div class="alert alert-block alert-info">

**TODO:**
* check all texts
* fix comments
* have it without outputs on Github
* more description to model building
* reminder on parameters 
* Add note on CV
* explain numbers in model generation
* make sure to make it possible to go through this in 1:00
</div>

# Deep Regression

Data - prepared vector data

Goal - use deep learning for predict the median income from zip code level population and spatial variables, assess the model accuracy with a test dataset, predicts the number to all zip codes and writes it to a geopackage

Content of this notebook:

0. Prepare environment
1. Set paths
2. Check for GPU
3. Reading and preparing data 
4. Model definition
5. Prediction and inference
6. Comparison to shallow




0. Prepare environment

In [2]:
import time
import pandas as pd
import geopandas as gpd
from math import sqrt
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split

2022-10-31 20:34:48.434491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 20:34:49.498321: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-31 20:34:50.051009: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-31 20:34:52.313600: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [3]:
random_seed= 42

1. Set paths

In [10]:
username = os.environ.get('USER')
base_directory= f'/scratch/project_2002044/{username}/2022/GeoML'

data_directory = os.path.join(base_directory,'data')

preprocessed_data_directory = os.path.join(data_directory,'preprocessed_regression')
train_dataset_name = os.path.join(preprocessed_data_directory,'scaled_train_zip_code_data.csv')
test_dataset_name = os.path.join(preprocessed_data_directory,'scaled_test_zip_code_data.csv')
val_dataset_name = os.path.join(preprocessed_data_directory,'scaled_val_zip_code_data.csv')
train_label_name = os.path.join(preprocessed_data_directory,'train_income_labels.pkl')
test_label_name = os.path.join(preprocessed_data_directory,'test_income_labels.pkl')
val_label_name = os.path.join(preprocessed_data_directory,'val_income_labels.pkl')

results_directory = os.path.join(data_directory,'regression_results')

metrics_filename = os.path.join(results_directory,'shallow_metrics.csv')

2. Check for GPUs

In [6]:
def checkGPUavailability():
    device = tensorflow.config.list_physical_devices('GPU')
    if device:
        print("We have a GPU available!")
    else:
        print("Sadly no GPU available. :( you have settle with a CPU. Good luck!")

checkGPUavailability()

Sadly no GPU available. :( you have settle with a CPU. Good luck!


2022-10-31 20:35:08.906771: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-31 20:35:08.906818: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (r18c48.bullx): /proc/driver/nvidia/version does not exist


3. Reading and preparing data 

In [17]:
# read train and test datasets
x_train = pd.read_csv(train_dataset_name)
x_val = pd.read_csv(val_dataset_name)
y_train = pd.read_pickle(train_label_name)
y_val = pd.read_pickle(val_label_name)
num_of_x_columns =  x_train.to_numpy().shape[1]


In [18]:
print(x_train)

       euref_x   euref_y  pinta_ala  he_vakiy  he_naiset  he_miehet   he_kika  \
0     0.449434  2.052769   0.484597 -0.478269  -0.471082  -0.485134  0.047259   
1     2.424721 -0.042865  -0.182227 -0.584315  -0.568128  -0.600758  2.806721   
2     0.789101  0.595826  -0.064490 -0.261986  -0.276989  -0.245149 -0.926668   
3     0.739835  3.708484   9.090226 -0.388017  -0.390725  -0.384300  0.534223   
4    -0.201411 -1.110719  -0.377133  1.004122   0.951233   1.059644 -1.900596   
...        ...       ...        ...       ...        ...        ...       ...   
1757  1.017255 -0.714784  -0.107610 -0.525007  -0.514969  -0.534879  0.696544   
1758 -0.070196  0.475040   0.822036 -0.184627  -0.211467  -0.155071 -0.115062   
1759  0.114911  0.364812   0.362871 -0.520816  -0.512496  -0.528829  0.696544   
1760 -1.308171 -1.166845  -0.266485 -0.510502  -0.499515  -0.521435  0.209581   
1761  1.061493 -0.331134  -0.276439 -0.564975  -0.553293  -0.576557  0.371902   

        he_0_2    he_3_6   

## 4. Model definition

In [23]:
# Initialize a Sequential model
model = Sequential()

# Add first layer with 64 perceptrons. Activation function is relu
model.add(Dense(64, activation='relu', input_shape=(num_of_x_columns,)))

# Add another layer with 64 perceptrons
model.add(Dense(64, activation='relu'))

# The last layer has to have only 1 perceptron as it is the output layer
model.add(Dense(1))

# Setting optimizer and loss functions. Learning rate set to 0.001
model.compile(optimizer=RMSprop(learning_rate=.001), loss='mse', metrics=['mae','mse'])
print(model.summary())

#Train the network with 1000 epochs and batch size of 64
model.fit(x_train, y_train, epochs=1000, shuffle=True, batch_size=64, verbose=2)



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                7552      
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 11,777
Trainable params: 11,777
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
28/28 - 0s - loss: 477805472.0000 - mae: 21598.2852 - mse: 477805472.0000 - 485ms/epoch - 17ms/step
Epoch 2/1000
28/28 - 0s - loss: 476389888.0000 - mae: 21565.9766 - mse: 476389888.0000 - 40ms/epoch - 1ms/step
Epoch 3/1000
28/28 - 0s - loss: 473636288.0000 - mae: 21502.7402 - mse: 473636288.000

## 5. Inference

In [24]:
#Evaluating the performance of the model using test data
prediction = model.predict(x_val)
r2 = r2_score(y_val, prediction)
rmse = sqrt(mean_squared_error(y_val, prediction))
mae = mean_absolute_error(y_val, prediction)

print("\nMODEL ACCURACY METRICS WITH TEST DATASET: \n" +
        "\t Root mean squared error: "+ str(rmse) + "\n" +
        "\t Mean absolute error: " + str(mae) + "\n" +
        "\t Coefficient of determination: " + str(r2) + "\n")


19/19 [==============================] - 0s 1ms/step

MODEL ACCURACY METRICS WITH TEST DATASET: 
	 Root mean squared error: 1461.4603948297417
	 Mean absolute error: 1020.1437676346891
	 Coefficient of determination: 0.7915513910338212



## 6. Comparison to shallow models and baseline 

In [22]:

shallow_metrics = pd.read_csv(metrics_filename)

print(shallow_metrics.sort_values(by=['RMSE'], ascending=False))

   Unnamed: 0                        model         RMSE          MAE        R2
0           0      Naive median prediction  3210.315973  2521.001704 -0.005820
6           6           AdaBoost Regressor  1745.176252  1363.223006  0.702763
1           1            Linear Regression  1600.219500  1178.917810  0.750090
2           2  Gradient Boosting Regressor  1522.333054  1152.671273  0.773825
3           3      Random Forest Regressor  1449.204208  1081.964225  0.795033
5           5            Bagging Regressor  1413.473238  1061.154174  0.805016
4           4        Extra Trees Regressor  1282.324966   950.631289  0.839520


## 7. Task

In [ ]:
Study the tensorflow documentation and experiment with different hyperparameter values. Can you improve on the accuracy or make the training faster?
Report the best results (on test set!) and mark down parameters used, so that others can reproduce the results. 